# NWS Raw Forecast Backtesting

Tests whether trading purely on the **raw NWS forecast** (no XGBoost correction) would have been profitable.

- `mu` = raw `forecasted_TMAX` from NWS (no model correction)
- `sigma` = std of historical NWS forecast errors (`TMAX_next_d - forecasted_TMAX`), computed over **all** available data (no leakage since no model is trained)
- Strategy: buy 1 NO contract per day on the market with the highest `edge_no_cents`

In [ ]:
import pandas as pd
import numpy as np
import math
import re

from model_copy import get_ev
from get_market_data import make_daily_request_table, fetch_daily_candles_from_table

# ── Paths ────────────────────────────────────────────────────────────────────
# Adjust these paths to match your local setup
BACKTESTING_CSV  = "/Users/giulioelmi/Desktop/kelshi_trading/backtesting/backtesting_data.csv"
SERIES_TICKER    = "KXHIGHLAX"
TARGET_HHMM      = "12:00"   # snapshot time for market prices (UTC)

## 1. Load data and compute NWS sigma

`y = TMAX_next_d - forecasted_TMAX` is the NWS forecast error already stored in the CSV.
We use **all** rows (train + backtest period) to compute sigma — no leakage because we never train a model.

In [ ]:
backtesting_data = pd.read_csv(BACKTESTING_CSV, index_col=0)
backtesting_data["DATE"] = pd.to_datetime(backtesting_data["DATE"])
backtesting_data = backtesting_data.sort_values("DATE").reset_index(drop=True)

print(f"Rows: {len(backtesting_data)}")
print(f"Date range: {backtesting_data['DATE'].min().date()} → {backtesting_data['DATE'].max().date()}")
backtesting_data[["DATE", "TMAX", "forecasted_TMAX", "TMAX_next_d", "y"]].head()

In [ ]:
# NWS forecast error distribution
forecast_errors = backtesting_data["y"].dropna()   # y = TMAX_next_d - forecasted_TMAX

SIGMA_NWS = forecast_errors.std(ddof=1)
mae_nws   = forecast_errors.abs().mean()

print(f"NWS forecast error — sigma (std): {SIGMA_NWS:.4f}°F")
print(f"NWS forecast error — MAE:         {mae_nws:.4f}°F")
print(f"(For reference, the XGBoost-corrected model sigma was ~2.53°F)")

forecast_errors.hist(bins=30, edgecolor="black")

## 2. Build the forecast results dataframe

Each row: `DATE` + `forecasted_TMAX` (= mu passed to `get_ev`).

In [ ]:
forecast_results = backtesting_data[["DATE", "forecasted_TMAX"]].copy()

data_start_date = forecast_results["DATE"].min().date()
data_end_date   = forecast_results["DATE"].max().date()

print(f"Forecast mu date range: {data_start_date} → {data_end_date}")
forecast_results.head()

## 3. Fetch Kalshi market prices

One snapshot per day at `12:00 UTC` — same methodology as the XGBoost backtesting notebook.

> **Note:** This makes ~1 API call per day. For ~150 days expect ~75 seconds (0.5s sleep per call).

In [ ]:
req = make_daily_request_table(
    series_ticker=SERIES_TICKER,
    start_date=data_start_date,
    end_date=data_end_date,
    target_hhmm=TARGET_HHMM,
)
req.head()

In [ ]:
market_prices = fetch_daily_candles_from_table(req)
print(f"Market rows fetched: {len(market_prices)}")
market_prices.head()

## 4. Add floor / cap columns

Infer bucket boundaries from ticker suffix (`B` = bucket, `T` = tail).

In [ ]:
def add_floor_cap(df, group_cols=("day", "event_ticker")):
    out = df.copy()
    out["floor"] = np.nan
    out["cap"]   = np.nan

    is_B = out["threshold_type"].eq("B")
    out.loc[is_B, "floor"] = out.loc[is_B, "threshold"] - 0.5
    out.loc[is_B, "cap"]   = out.loc[is_B, "threshold"] + 0.5

    def _assign_tails(g):
        g = g.copy()
        t_idx = g.index[g["threshold_type"].eq("T")]
        if len(t_idx) == 0:
            return g

        b = g[g["threshold_type"].eq("B")]
        if not b.empty:
            min_b_floor = (b["threshold"] - 0.5).min()
            max_b_cap   = (b["threshold"] + 0.5).max()
            for i in t_idx:
                thr = float(g.loc[i, "threshold"])
                if thr <= min_b_floor + 1e-9:
                    g.loc[i, "floor"] = np.nan
                    g.loc[i, "cap"]   = thr
                elif thr >= max_b_cap - 1e-9:
                    g.loc[i, "floor"] = thr
                    g.loc[i, "cap"]   = np.nan
            return g

        t_vals = g.loc[t_idx, "threshold"].astype(float)
        t_min, t_max = t_vals.min(), t_vals.max()
        g.loc[t_idx[t_vals.eq(t_min)], "cap"]   = t_min
        g.loc[t_idx[t_vals.eq(t_max)], "floor"] = t_max
        return g

    out = out.groupby(list(group_cols), group_keys=False).apply(_assign_tails)
    return out


markets_df = add_floor_cap(market_prices, group_cols=("day", "event_ticker"))
markets_df["no_ask"] = (1 - markets_df["yes_ask"]).round(4)
markets_df.head()

## 5. Compute EV for every day using raw NWS forecast as mu

In [ ]:
def compute_daily_evs(
    markets_df: pd.DataFrame,
    results_df: pd.DataFrame,
    sigma: float,
    day_col_markets: str = "day",
    day_col_results: str = "DATE",
    mu_col_results: str = "forecasted_TMAX",
) -> pd.DataFrame:
    markets = markets_df.copy()
    results = results_df.copy()

    markets[day_col_markets] = pd.to_datetime(markets[day_col_markets]).dt.date
    results[day_col_results] = pd.to_datetime(results[day_col_results]).dt.date

    for c in ["floor", "cap", "yes_ask", "no_ask"]:
        markets[c] = pd.to_numeric(markets[c], errors="coerce")
    results[mu_col_results] = pd.to_numeric(results[mu_col_results], errors="coerce")

    markets = markets.dropna(subset=[day_col_markets, "yes_ask", "no_ask"])

    day_mu = (
        results.dropna(subset=[mu_col_results])
               .sort_values(day_col_results)
               .drop_duplicates(subset=[day_col_results], keep="last")
               .set_index(day_col_results)[mu_col_results]
    )

    outputs = []
    for day, mk_day in markets.groupby(day_col_markets, sort=True):
        if day not in day_mu.index:
            continue
        mu = float(day_mu.loc[day])
        out_day = get_ev(mk_day, mu=mu, sigma=float(sigma))
        out_day["mu"]    = mu
        out_day["sigma"] = float(sigma)
        outputs.append(out_day)

    if not outputs:
        return pd.DataFrame()
    return pd.concat(outputs, axis=0, ignore_index=True)


ev_df = compute_daily_evs(
    markets_df=markets_df,
    results_df=forecast_results,
    sigma=SIGMA_NWS,
    day_col_markets="day",
    day_col_results="DATE",
    mu_col_results="forecasted_TMAX",
)

print(f"EV rows: {len(ev_df)}")
ev_df.head()

## 6. Select best NO bet per day

In [ ]:
df_max = ev_df.loc[ev_df.groupby("event_ticker")["edge_no_cents"].idxmax()]

df_max_final = (
    df_max[["day", "threshold", "edge_no_cents", "floor", "cap", "no_ask"]]
    .rename(columns={"day": "DATE"})
    .copy()
)
df_max_final["DATE"] = pd.to_datetime(df_max_final["DATE"])
df_max_final.head()

## 7. Merge with actual TMAX and compute settlement

**Settlement rule for NO:**
- Bucket `[floor, cap)`: NO wins when `TMAX < floor` OR `TMAX >= cap`
- Upper tail (floor only): NO wins when `TMAX < floor`
- Lower tail (cap only): NO wins when `TMAX >= cap`

In [ ]:
temperature = backtesting_data[["DATE", "TMAX"]].copy()
temperature["DATE"] = pd.to_datetime(temperature["DATE"])

df_pnl = df_max_final.merge(temperature, on="DATE")
df_pnl["fee"] = 0.02

def no_wins(row):
    """Returns 1 if the NO contract settles to $1, else 0."""
    floor, cap, tmax = row["floor"], row["cap"], row["TMAX"]
    has_floor = pd.notna(floor)
    has_cap   = pd.notna(cap)
    if has_floor and has_cap:       # bucket: YES wins if floor <= TMAX < cap
        return int(not (floor <= tmax < cap))
    elif has_floor:                 # upper tail: YES wins if TMAX >= floor
        return int(tmax < floor)
    elif has_cap:                   # lower tail: YES wins if TMAX < cap
        return int(tmax >= cap)
    return 0

df_pnl["outcome"] = df_pnl.apply(no_wins, axis=1)
df_pnl["pnl_1_contract"] = df_pnl["outcome"] - df_pnl["no_ask"] - df_pnl["fee"]

df_pnl.head(20)

## 8. Results — buy every day (no edge filter)

In [ ]:
profit           = df_pnl["pnl_1_contract"].sum()
capital_deployed = df_pnl["no_ask"].sum()
roi              = profit / capital_deployed * 100
mean_pnl         = df_pnl["pnl_1_contract"].mean()
sd_pnl           = df_pnl["pnl_1_contract"].std()
avg_edge         = df_pnl["edge_no_cents"].mean()
pct_correct      = round(df_pnl["outcome"].sum() / len(df_pnl) * 100)
n_bets           = len(df_pnl)

print(f"{'Bets placed':<28} {n_bets}")
print(f"{'Total P&L':<28} ${profit:.2f}")
print(f"{'Capital deployed':<28} ${capital_deployed:.2f}")
print(f"{'Return on capital':<28} {roi:.1f}%")
print(f"{'Mean P&L per bet':<28} ${mean_pnl:.4f}")
print(f"{'Std P&L per bet':<28} ${sd_pnl:.4f}")
print(f"{'Win rate':<28} {pct_correct}%")
print(f"{'Average edge':<28} {avg_edge:.4f}")
print(f"{'NWS sigma used':<28} {SIGMA_NWS:.4f}°F")

## 9. Edge-filter sensitivity

Only bet when `edge_no_cents > threshold`.

In [ ]:
EDGE_THRESHOLD = 0.35

df_filtered = df_pnl[df_pnl["edge_no_cents"] > EDGE_THRESHOLD].copy()
df_filtered["pnl_filtered"] = df_filtered["outcome"] - df_filtered["no_ask"] - df_filtered["fee"]

profit_f      = df_filtered["pnl_filtered"].sum()
capital_f     = df_filtered["no_ask"].sum()
roi_f         = profit_f / capital_f * 100 if capital_f > 0 else float("nan")
pct_correct_f = round(df_filtered["outcome"].sum() / len(df_filtered) * 100) if len(df_filtered) > 0 else 0
avg_edge_f    = df_filtered["edge_no_cents"].mean()

print(f"Edge filter > {EDGE_THRESHOLD}")
print(f"{'Bets placed':<28} {len(df_filtered)}")
print(f"{'Total P&L':<28} ${profit_f:.2f}")
print(f"{'Capital deployed':<28} ${capital_f:.2f}")
print(f"{'Return on capital':<28} {roi_f:.1f}%")
print(f"{'Win rate':<28} {pct_correct_f}%")
print(f"{'Average edge':<28} {avg_edge_f:.4f}")

## 10. Cumulative P&L over time

In [ ]:
import matplotlib.pyplot as plt

df_plot = df_pnl.sort_values("DATE").copy()
df_plot["cumulative_pnl"] = df_plot["pnl_1_contract"].cumsum()

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(df_plot["DATE"], df_plot["cumulative_pnl"], label="NWS forecast (no model correction)")
ax.axhline(0, color="black", linewidth=0.8, linestyle="--")
ax.set_title("Cumulative P&L — NWS raw forecast strategy (1 contract/day, best NO edge)")
ax.set_xlabel("Date")
ax.set_ylabel("Cumulative P&L ($)")
ax.legend()
plt.tight_layout()
plt.show()